In [1]:
import numpy as np
import pandas as pd
import tracktor as tr
import collections
import cv2
import sys
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [2]:
# s-w-w-s-w-s-s-w
colours = [(100,100,255),(100,255,255),(255,100,255),(200,200,200),(255,255,100),(255,100,100),(100,255,100),(100,100,100)]
#colours = [(100,100,255),(100,255,255),(255,100,255),(200,200,200),(255,255,100),(255,100,100),(100,255,100),(100,100,100)]
colours = [(100,100,255),(255,100,100),(180,30,30),(30,30,180),(100,255,100),(30,180,180),(100,255,255),(30,180,30)]
colours = [(126,255,255),(18,255,255),(36,255,255),(54,255,255),(72,255,255),(90,255,255),(108,255,255),(162,255,255)]

frame_range = 144
max_radius = 5

df = pd.read_csv("/Users/vivekhsridhar/Documents/Code/Python/OpenCV/tracktor/output/spider_video_tracked.csv")
input_vidpath = '/Users/vivekhsridhar/Documents/Code/Python/OpenCV/tracktor/videos/spider_video.mp4'
output_vidpath = '/Users/vivekhsridhar/Documents/Code/Python/OpenCV/tracktor/output/spider_video_tracked.mp4'
codec = 'DIVX'

In [3]:
for idx, ID in enumerate(np.unique(df['id'])):
    df['id'][df['id'] == ID] = idx

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
## Open video
cap = cv2.VideoCapture(input_vidpath)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]),int(cap.read()[1].shape[0]))
out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 60.0, frameSize = output_framesize, isColor = True)

last = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    
    tmp = df[(df['frame'] > this-frame_range) & (df['frame'] <= this)]
    tmp = tmp.reset_index()
    if ret == True:
        # Preprocess the image for background subtraction
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(thresh, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 501, 20)
        
        for i in range(0,len(tmp)):
            x = int(tmp.loc[i,'pos_x'])
            y = int(tmp.loc[i,'pos_y'])
            r = int(max_radius - (this - tmp.loc[i,'frame'])//(frame_range/max_radius))
            c = colours[int(tmp.loc[i,'id'])]
            cv2.circle(frame, (x,y), r, c, -1, cv2.LINE_AA)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
        
        # Display the resulting frame
        out.write(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 27:
            break
            
    if last >= this:
        break
    
    last = this

## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1